In [ ]:
# import dependencies
# import sys
# !{sys.executable} -m pip install seaborn
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import glacierml as gl
import numpy as np
import warnings
from tensorflow.python.util import deprecation
import os
import logging
import seaborn as sns
from tqdm import tqdm
from IPython.display import display, HTML
import geopy
# display(HTML("<style>.container { width:85% !important; }</style>"))
tf.get_logger().setLevel(logging.ERROR)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
deprecation._PRINT_DEPRECATION_WARNINGS = False
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
pd.set_option('display.max_columns', None)

pd.set_option('mode.chained_assignment', None)

RS = range(0,25,1)

print('currently running tensorflow version: ' + tf.__version__)
# df = df[[
#     'area_g',
#     'Area'
# ]]


In [ ]:
# select dataset to analyze

selected_dataset = 'df6'

if selected_dataset == 'df1':
    df1 = gl.data_loader(
#         root_dir = '/home/simonhans/data/prethicktor/',
        RGI_input = 'n'
    )
    module = 'sm1'
    res = 'sr1'
    dataset = df1
    dataset.name = selected_dataset
if selected_dataset == 'df2':
    df2 = gl.data_loader(
#         root_dir = '/home/simonhans/data/prethicktor/',
        RGI_input = 'y',
        scale = 'g',
        area_scrubber = 'off'
    )
    module = 'sm2'
    res = 'sr2'
    dataset = df2
    dataset.name = selected_dataset
if selected_dataset == 'df3':
    df3 = gl.data_loader(
#         root_dir = '/home/simonhans/data/prethicktor/',
        RGI_input = 'y',
        scale = 'g',
        area_scrubber = 'on',
        anomaly_input = 1
    )
    module = 'sm3'
    res = 'sr3'
    dataset = df3
    dataset.name = selected_dataset
if selected_dataset == 'df4':
    df4 = gl.data_loader(
#         root_dir = '/home/simonhans/data/prethicktor/',
        RGI_input = 'y',
        scale = 'g',
        area_scrubber = 'on',
        anomaly_input = 5
    )
    module = 'sm4'
    res = 'sr4'
    dataset = df4
    dataset.name = selected_dataset
if selected_dataset == 'df5':
    df5 = gl.data_loader(
#         root_dir = '/home/simonhans/data/prethicktor/',
        RGI_input = 'y',
        scale = 'g',
        area_scrubber = 'off'
    )
    df5 = df5.drop('Zmed', axis = 1)
    dataset = df5
    dataset.name = selected_dataset
    module = 'sm5'
    res = 'sr5'
if selected_dataset == 'df6':
    df6 = gl.data_loader(
#         root_dir = '/home/simonhans/data/prethicktor/',
        RGI_input = 'y',
        scale = 'r',
        region_selection = 1,
        area_scrubber = 'off'
    )
    module = 'sm6'
    res = 'sr6'
    reg = df6['region'].iloc[-1]
    df6 = df6.drop('region', axis=1)
    dataset = df6 
    dataset.name = str('df6_' + str(reg))

In [ ]:
rootdir = 'zults/'
predictions = pd.DataFrame()
deviations = pd.DataFrame()
for file in tqdm(os.listdir(rootdir)):
    if 'predictions' in file:
        file_reader = pd.read_csv(rootdir + file)
        predictions = predictions.append(file_reader, ignore_index = True)
    if 'deviations' in file:
        file_reader = pd.read_csv(rootdir + file)
        deviations = pd.concat([deviations, file_reader], ignore_index = True)
        
deviations = deviations.drop('Unnamed: 0', axis = 1)
predictions = predictions.drop('Unnamed: 0', axis = 1)
deviations['total parameters'] = deviations['total parameters'].astype(int)
deviations['trained parameters'] = deviations['trained parameters'].astype(int)
deviations['total inputs'] = deviations['total inputs'].astype(int)
deviations = deviations[
    (deviations['df'].str.contains('df6')) 
#     &
#     (deviations['layer architecture'] == '24-12')
#     &
#     (deviations['learning rate'] == 0.010)
#     &
#     (deviations['epochs'] == 100)
]
deviations['test - train'] = (
    abs(deviations['test mae avg'] - deviations['train mae avg'])
)
deviations = deviations.sort_values(
    [
#         'test - train',
        'test mae avg', 
#         'test predicted thickness std dev'
#         'layer architecture',
#         'learning rate',
        'df'
    ]
)
deviations

In [ ]:
total_inputs = (len(dataset) * (len(dataset.columns) -1))
print('total inputs = ' + str(total_inputs))

non_trainable_parameters = (len(dataset.columns)) + ((len(dataset.columns) - 1))
print('non-trainable parameters = ' + str(non_trainable_parameters))

layer_1 = 50
layer_2 = 25

total_parameters = (
    layer_1 * (inputs + 1)  + 
    (layer_2 * (layer_1 + 1)) + 
    ((1 * (layer_2 + 1))) +
    non_trainable_parameters
)
print('trainable parameters = ' + str(total_parameters - non_trainable_parameters))

In [ ]:
"""
Here we load top rated model and look at predicted accuracies of each random state
"""
print('Please select index from deviations table to inspect further')

selection = int(input())

arch = deviations['layer architecture'].loc[selection]
top_learning_rate = deviations['learning rate'].loc[selection]
epochs = deviations['epochs'].loc[selection]
dropout = deviations['dropout'].loc[selection]
print(arch)
print(top_learning_rate)
print(epochs)
print(dropout)

dnn_model = {}
rootdir = 'saved_models/' + module + '/sm_' + arch + '/'


train_features, test_features, train_labels, test_labels = gl.data_splitter(dataset)
features = pd.concat([train_features, test_features], ignore_index = True)
labels = pd.concat([train_labels, test_labels], ignore_index = True)
for rs in tqdm(RS):
    
    model_name = (
        str(arch) + 
        '_' + 
        dataset.name +
        '_' + 
        str(dropout) +
        '_dnn_MULTI_' + 
        str(top_learning_rate) + 
        '_0.2_' +
        str(epochs) + 
        '_' + 
        str(rs)
    )   
    
    model_path = (
        rootdir + 
        dataset.name +
        '_' + 
        str(dropout) +
        '_dnn_MULTI_' + 
        str(top_learning_rate) + 
        '_0.2_' +
        str(epochs) + 
        '_' + 
        str(rs)
    )
    
    dnn_model[model_name] = tf.keras.models.load_model(model_path)
    
    
    y = dnn_model[model_name].predict(features, verbose = 0)
    fig,ax=plt.subplots(1,1,figsize=(15,10))
    fig.patch.set_facecolor('w')
    plt.plot(labels,y,'.')
    plt.plot((0,400),(0,400),'-')
    plt.xlabel('True Thickness (m)')
    plt.ylabel('Model Thickness (m)')
    ax.set_title('Random State ' +str(rs))
    plt.xlim((0,400))
    plt.ylim((0,400))

In [ ]:

fig_dir = (
    'figs/' + 
    module + 
    '/'   
) 
isdir = os.path.isdir(fig_dir)
if isdir == False:
    os.makedirs(fig_dir)

print(arch)
print(top_learning_rate)
print(epochs)
print(dropout)

print(fig_dir)
rootdir = 'saved_results/' + res + '/sr_' + arch + '/'
print(rootdir)
dnn_history = {}
for rs in RS: 
    
    history_name = (
        arch + 
        '_' +
        dataset.name +
        '_' +
        str(dropout) +
        '_dnn_history_MULTI_' +
        str(top_learning_rate) +
        '_0.2_' +
        str(epochs) + 
        '_' + 
        str(rs)
    )
    
    model_name = (
        dataset.name +
        '_' +
        str(dropout) +
        '_dnn_history_MULTI_' +
        str(top_learning_rate) +
        '_0.2_' +
        str(epochs) + 
        '_' + 
        str(rs)
    )
    
    dnn_history[model_name] = pd.read_csv(rootdir + model_name)

    fig,ax=plt.subplots(1,1,figsize=(10,10))
    fig.patch.set_facecolor('w')
    ax.set_title(history_name)
    gl.plot_loss(dnn_history[model_name])
    
file_name = (
    fig_dir +
    arch + '_' +
    dataset.name + '_' +
    str(dropout) + '_' +
    str(top_learning_rate) + '_' +
    str(epochs) + '_' +
    str(rs) + '_' +
    '.eps'
)

print(file_name)
fig.savefig(file_name)

In [ ]:
ccccombo_breaker()
# past this point is under construction. 
# here be monsters

In [ ]:
# compare RGI regional attributes with RGI regions matched with GlaThiDa thicknesses

dfs = pd.DataFrame()
pth_3 = '/data/fast1/glacierml/data/regional_data/raw/'
for file in tqdm(os.listdir(pth_3)):
    df = pd.read_csv(pth_3 + file, encoding_errors = 'replace', on_bad_lines = 'skip')
    region_and_number = file[:-4]
    region_number = region_and_number[:2]
    region = region_and_number[3:]

    df['geographic region'] = region
    df['region'] = region_number
    dfs = dfs.append(df, ignore_index=True)

dfs = dfs.reset_index()

dfs = dfs[[
    'GlaThiDa_index',
    'RGI_index',
    'RGIId',
    'region',
    'geographic region'
]]

pth = '/data/fast1/glacierml/data/RGI/rgi60-attribs/'
RGI_extra = pd.DataFrame()
for file in os.listdir(pth):
    f = pd.read_csv(pth + file, encoding_errors = 'replace', on_bad_lines = 'skip')
    RGI_extra = pd.concat([RGI_extra, f], ignore_index = True)

    region_and_number = file[:-4]
    region_number = region_and_number[:2]
    region = region_and_number[9:]
    df = dfs[dfs['region'] == region_number]

    percent_trainable = (len(df) / len(f)) * 100

    print(
        'region ' + str(region_number) + ' has ' + str(len(f)) + ' lines of data, ' +
        str(len(df)) + ' or ' + str(percent_trainable) + '%'
        ' of which are trainable with GlaThiDa thicknesses'    
    )

In [ ]:
# load prethicktions
tau_b = 10*5
rho = 0.9167
g = 9.81

root_dir = 'zults/'
RGI_predicted = pd.DataFrame()

# read each prediction file and do stuff to it and append it to a table
for file in os.listdir(root_dir):
    if 'RGI_predicted' in file:
        file_reader = pd.read_csv(root_dir + file)
        
        file_reader['volume'] = (
            file_reader['avg predicted thickness'] / 1e3
        ) * file_reader['Area']
        
        # have to turn something into a series and append it to the df to build it
        sum_volume = sum(file_reader['volume'])
        total_volume = pd.Series(sum_volume, name = 'total volume')
        RGI_predicted = pd.concat([RGI_predicted, total_volume], ignore_index = True)
        
        # variance
        file_reader['variance'] = file_reader['predicted thickness std dev'] **2 
        variance = sum(file_reader['variance'])
        RGI_predicted.loc[
            RGI_predicted.index[-1], 'total variance'
        ] = np.sqrt(variance)/1e3

        # mean thickness
        thickness_mean = file_reader['avg predicted thickness'].mean()
        RGI_predicted.loc[
            RGI_predicted.index[-1], 'mean thickness (km)'
        ] = thickness_mean/1e3
        
#         # max thickness
#         thickness_max = file_reader['avg predicted thickness'].max()
#         RGI_predicted.loc[
#             RGI_predicted.index[-1], 'max model thickness (km)'
#         ] = thickness_max/1e3
        
        # area * mean thickness
        area = sum(file_reader['Area'])
        RGI_predicted.loc[
            RGI_predicted.index[-1], 'vol (10**3 km**3)'
        ] = (area * (thickness_mean/1e3))/1e3
        
        RGI_predicted.loc[
            RGI_predicted.index[-1], 'area'
        ] = area
        
        
# #         df1 has no lmax, so deriving thickness will not work
#         if 'df1' not in file:
#             file_reader['derived h max'] = (
#                 np.sqrt((2 * tau_b) / (rho * g)) * (file_reader['Lmax']/1e3)
#             )

#             file_reader['derived h avg'] = (
#                 (2/3) * file_reader['derived h max']
#             )
            
            
#             derived_h_max_sum = sum(file_reader['derived h max'])
#             RGI_predicted.loc[
#                 RGI_predicted.index[-1], 'derived h max (m)'
#             ] = derived_h_max_sum/1e3
            
            
#             derived_h_avg_sum = sum(file_reader['derived h avg'])
#             RGI_predicted.loc[
#                 RGI_predicted.index[-1], 'derived h avg (m)'
#             ] = derived_h_avg_sum/1e3
            
        if 'df1' in file:
            RGI_predicted.loc[
                RGI_predicted.index[-1], 'dataframe'
            ] = 'df1'
            
            if '10-5' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '10-5'
            if '16-8' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '16-8'
            if '24-12' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '24-12'
        if 'df2' in file:
            RGI_predicted.loc[
                RGI_predicted.index[-1], 'dataframe'
            ] = 'df2'   
            
            if '10-5' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '10-5'
            if '50-28' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '50-28'
            if '64-48' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '64-48'
        if 'df3' in file:
            RGI_predicted.loc[
                RGI_predicted.index[-1], 'dataframe'
            ] = 'df3'        
            
            if '10-5' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '10-5'
            if '37-20' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '37-20'
            if '59-28' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '59-28'
        if 'df4' in file:
            RGI_predicted.loc[
                RGI_predicted.index[-1], 'dataframe'
            ] = 'df4'    
            
            if '10-5' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '10-5'
                
            if '47-21' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '47-21'
                
            if '64-36' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '64-36'
        if 'df5' in file:
            RGI_predicted.loc[
                RGI_predicted.index[-1], 'dataframe'
            ] = 'df5'    
            
            if '10-5' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '10-5'
                
            if '50-25' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '50-25'
                
            if '64-42' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '64-42'
        if 'df6' in file:
            if 'df6_1' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'dataframe'
                ] = 'df6_1'
            
            if 'df6_2' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'dataframe'
                ] = 'df6_2'
                
            if 'df6_3' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'dataframe'
                ] = 'df6_3'
            
            if 'df6_6' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'dataframe'
                ] = 'df6_6'
            
            
            if 'df6_7' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'dataframe'
                ] = 'df6_7'
                
            if 'df6_8' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'dataframe'
                ] = 'df6_8'
                
            if 'df6_9' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'dataframe'
                ] = 'df6_9'
                
            if 'df6_10' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'dataframe'
                ] = 'df6_10'
                
            if 'df6_11' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'dataframe'
                ] = 'df6_11'
                
            if 'df6_13' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'dataframe'
                ] = 'df6_13'
            if 'df6_19' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'dataframe'
                ] = 'df6_19'
                
                
            if '10-5' in file:
                RGI_predicted.loc[
                    RGI_predicted.index[-1], 'architecture'
                ] = '10-5'
                
#         if '0.1' in file:
#             RGI_predicted.loc[RGI_predicted.index[-1], 'learning rate']= '0.100'
#         if '0.01' in file:
#             RGI_predicted.loc[RGI_predicted.index[-1], 'learning rate']= '0.010'
#         if '0.001' in file:
#             RGI_predicted.loc[RGI_predicted.index[-1], 'learning rate']= '0.001'
            
#         if '_20' in file:
#             RGI_predicted.loc[RGI_predicted.index[-1], 'epochs']= '20'
#         if '_25' in file:
#             RGI_predicted.loc[RGI_predicted.index[-1], 'epochs']= '25'
#         if '_50' in file:
#             RGI_predicted.loc[RGI_predicted.index[-1], 'epochs']= '50'
#         if '_60' in file:
#             RGI_predicted.loc[RGI_predicted.index[-1], 'epochs']= '60'
#         if '_15' in file:
#             RGI_predicted.loc[RGI_predicted.index[-1], 'epochs']= '15'
#         if '_30' in file:
#             RGI_predicted.loc[RGI_predicted.index[-1], 'epochs']= '30'
#         if '_40' in file:
#             RGI_predicted.loc[RGI_predicted.index[-1], 'epochs']= '40'
#         if '_100' in file:
#             RGI_predicted.loc[RGI_predicted.index[-1], 'epochs']= '100'

RGI_predicted = RGI_predicted.drop(0, axis=1)

RGI_predicted = RGI_predicted[
    (RGI_predicted['dataframe'].str.contains('df')) 
#     &
#     (RGI_predicted['layer architecture'] == '24-12')
#     &
#     (RGI_predicted['learning rate'] == 0.010)
#     &
#     (RGI_predicted['epochs'] == 100)
]

RGI_predicted.sort_values([
    'mean thickness (km)',
#     'architecture',
#     'learning rate',
#     'dataframe'
], ascending = True)

RGI_predicted = RGI_predicted[
    RGI_predicted['vol (10**3 km**3)'] < 300
]
RGI_predicted

In [ ]:
df = gl.data_loader

In [ ]:
"""
This is a density plot of the most recent predictions made. Can easily be modified in a loop
to show multiple random states and whatnot
"""
sns.set(rc={"figure.figsize":(15,10)})
sns.kdeplot(x = test_labels, y = y.flatten(),fill = True)
plt.plot((0,300),(0,300),'-')

In [ ]:
"""
This cell computes the true average thickness of the glaciers in use
"""


pth = '/data/fast1/glacierml/T_models/'
T_lab = pd.read_csv(pth + 'T.csv', low_memory = False)
T_lab = T_lab[[
    'GlaThiDa_ID',
    'LAT',
    'LON',
    'AREA',
    'MEAN_SLOPE',
    'MEAN_THICKNESS'
]]
T_lab = T_lab.dropna()

tru_thickness = np.sum(T_lab['MEAN_THICKNESS']) / len(T_lab['MEAN_THICKNESS'])
tru_thickness